In [ ]:
SERVER_PORT=49052
SERVER_HOST='0.0.0.0'
DATABASE_PORT=27017

import datetime
from bson.objectid import ObjectId
import random
import string
import hmac
import hashlib
from hashlib import sha256
from flask import Flask
from flask import request,redirect
from flask import Response
from flask import render_template
from os import listdir
from os.path import isfile, join
import os
from pymongo import MongoClient
import pymongo
import os.path
from base64 import b64encode
from collections import OrderedDict
from hashlib import sha256
from hmac import HMAC
from urllib.parse import urlparse, parse_qsl, urlencode
import json

app = Flask(__name__)
app.config['TEMPLATES_AUTO_RELOAD'] = True

@app.after_request
def add_header(r):
    r.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    r.headers["Pragma"] = "no-cache"
    r.headers["Expires"] = "0"
    r.headers['Cache-Control'] = 'public, max-age=0'
    return r

@app.route('/',methods=["POST"])
def command():
    t=request.json
    client = MongoClient(port=DATABASE_PORT)
    db=client.srp
    appID=t["appID"]
    appObject=db.apps.find_one({"_id":ObjectId(appID)})
    tSign=b64encode(HMAC(appObject["secretKey"].encode(), t["commandHash"].encode(), sha256).digest())
    if t["appURL"]!=appObject["appSite"]:
        return "False"
    if tSign.decode("utf-8")==t["serverSign"]:
        allowedCommands=list()
        for i in ["list","read","write","delete","mkdir","rename"]:
            if appObject[i]=="on":
                allowedCommands.append(i)
        return str(t["type"] in allowedCommands)
    else:
        return "False"

app.run(host=SERVER_HOST, port=SERVER_PORT, threaded=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:49052/ (Press CTRL+C to quit)
